In [ ]:
  !pip install wandb


In [ ]:
import wandb


In [ ]:
!wandb login 58a0b576fd5221cd0d63b154deaabbe535e853c6

CHARACTER LEVEL WITHOUT ATTENTION

In [4]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'embedding_dim': {'values': [64,128,256]},
        'hidden_dim': {'values': [64, 128,256 ]},
        'enc_layers': {'values': [1, 2,3]},
        'dec_layers': {'values': [1, 2,3]},
        'cell_type': {'values': ['GRU', 'LSTM', 'RNN']},
        'dropout': {'values': [0.2, 0.3,0.5]},
        'epochs': {'values': [20, 15]},
        'beam_size': {'values': [1, 3, 5]},
        'batch_size': {'values': [64, 128, 256]},
        'learning_rate': {'values': [0.001, 0.0005, 0.0001]}
    }
}

default_config = {
    'embedding_dim': 32,
    'hidden_dim': 64,
    'enc_layers': 1,
    'dec_layers': 1,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 10,
    'beam_size': 1,
    'batch_size': 64,
    'learning_rate': 0.001
}


# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        inp_vocab.build([p[0] for p in self.pairs])
        out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Encoder and Decoder
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

# =======================
# Seq2Seq Model with Beam Search
# =======================
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])

        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)

        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Train Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Val Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

# =======================
# Main
# =======================
def main():
    wandb.init(config=default_config, project="dakshina-transliteration")
    config = wandb.config
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
    dev_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
    train_data = TransliterationDataset(train_path, inp_vocab, out_vocab)
    dev_data = TransliterationDataset(dev_path, inp_vocab, out_vocab)
    
    # Use config.batch_size for DataLoader
    train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim, config.enc_layers, config.cell_type, config.dropout)
    decoder = Decoder(out_vocab.size, config.embedding_dim, config.hidden_dim, config.dec_layers, config.cell_type, config.dropout)
    model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers, config.cell_type, device).to(device)
    
    # Use config.learning_rate for optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(config.epochs):
        print(f"Epoch {epoch+1}/{config.epochs}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "epoch": epoch+1})

# =======================

if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="dakshina-translit")
    wandb.agent(sweep_id, function=main,count=30)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: q6l1non5
Sweep URL: https://wandb.ai/manglesh_dl_ass3/dakshina-transliteration/sweeps/q6l1non5


wandb: Agent Starting Run: t64a62c3 with config:
wandb: 	beam_size: 5
wandb: 	cell_type: RNN
wandb: 	dec_layers: 2
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 128
wandb: 	enc_layers: 3
wandb: 	epochs: 20
wandb: 	hidden_dim: 256
wandb: Currently logged in as: manglesh_dlass3 (manglesh_dl_ass3) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/20
Train Loss: 1.9554, Acc: 42.42%
Val Loss: 1.7184, Acc: 46.60%
Epoch 2/20
Train Loss: 1.5413, Acc: 52.60%
Val Loss: 1.5627, Acc: 50.22%
Epoch 3/20
Train Loss: 1.3921, Acc: 56.85%
Val Loss: 1.4492, Acc: 53.92%
Epoch 4/20
Train Loss: 1.3088, Acc: 59.20%
Val Loss: 1.3864, Acc: 55.18%
Epoch 5/20
Train Loss: 1.2589, Acc: 60.54%
Val Loss: 1.3533, Acc: 56.45%
Epoch 6/20
Train Loss: 1.2227, Acc: 61.63%
Val Loss: 1.3185, Acc: 57.52%
Epoch 7/20
Train Loss: 1.1909, Acc: 62.43%
Val Loss: 1.2978, Acc: 57.83%
Epoch 8/20
Train Loss: 1.1754, Acc: 62.80%
Val Loss: 1.3139, Acc: 57.95%
Epoch 9/20
Train Loss: 1.1584, Acc: 63.26%
Val Loss: 1.3047, Acc: 58.31%
Epoch 10/20
Train Loss: 1.1579, Acc: 63.28%
Val Loss: 1.2658, Acc: 58.91%
Epoch 11/20
Train Loss: 1.1431, Acc: 63.65%
Val Loss: 1.2666, Acc: 58.62%
Epoch 12/20
Train Loss: 1.1346, Acc: 63.93%
Val Loss: 1.2665, Acc: 59.07%
Epoch 13/20
Train Loss: 1.1267, Acc: 64.30%
Val Loss: 1.2796, Acc: 58.97%
Epoch 14/20
Train Loss: 1.1270, Acc: 64.23%
Val

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_acc,▁▄▆▆▇▇▇▇████████████
train_loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▅▆▆▇▇▇▇█▇██▇████▇█
val_loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,20
train_acc,64.44335
train_loss,1.11939
val_acc,58.94795
val_loss,1.27341


...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇▇▇█████████
train_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▅▅▆▆▇▇▇▇██▇▇██
val_loss,█▄▄▂▃▃▂▃▂▁▂▂▃▂▂
epoch,15
train_acc,81.38486
train_loss,0.56288
val_acc,74.55224
val_loss,0.80338


wandb: Agent Starting Run: 1zswwpkt with config:
wandb: 	beam_size: 5
wandb: 	cell_type: GRU
wandb: 	dec_layers: 1
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 64
wandb: 	enc_layers: 2
wandb: 	epochs: 20
wandb: 	hidden_dim: 256


Epoch 1/20


wandb: Ctrl + C detected. Stopping sweep.


Train Loss: 1.2082, Acc: 62.37%
Val Loss: 1.0177, Acc: 66.39%
Epoch 2/20
Train Loss: 0.7004, Acc: 77.47%
Val Loss: 0.9014, Acc: 70.19%
Epoch 3/20
Train Loss: 0.6098, Acc: 80.10%
Val Loss: 0.8823, Acc: 71.00%
Epoch 4/20


**TESTING without Attention**

In [7]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import random
import numpy as np

# =======================
# Fixed Best Configuration
# =======================
best_config = {
    'embedding_dim': 128,
    'hidden_dim': 256,
    'enc_layers': 3,
    'dec_layers': 3,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 15,
    'beam_size': 5,
    'batch_size': 128,  # Added
    'learning_rate': 0.001  # Added
}


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.init(project="dakshina-transliteration", config=config)
config = wandb.config

# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab=None, out_vocab=None):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        if inp_vocab and out_vocab:
            inp_vocab.build([p[0] for p in self.pairs])
            out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Encoder / Decoder / Seq2Seq
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)

        enc_hidden = self.encoder(src[0], src[1])

        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)

        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if random.random() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

# =======================
# Train / Evaluate
# =======================
def compute_accuracy(preds, targets):
    preds = preds.argmax(-1)
    correct = ((preds == targets) & (targets != 0)).sum().item()
    total = (targets != 0).sum().item()
    return correct / total

def train_eval(model, loader, criterion, optimizer, is_train):
    model.train() if is_train else model.eval()
    total_loss, total_acc = 0, 0
    with torch.set_grad_enabled(is_train):
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            if is_train: optimizer.zero_grad()
            output = model((src, src_lens), trg)
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
            acc = compute_accuracy(output[:, 1:], trg[:, 1:])
            if is_train:
                loss.backward()
                optimizer.step()
            total_loss += loss.item()
            total_acc += acc
    return total_loss / len(loader), total_acc / len(loader)

# =======================
# Train and Save Best
# =======================
inp_vocab, out_vocab = Vocab(), Vocab()
train_set = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv", inp_vocab, out_vocab)
dev_set = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv", inp_vocab, out_vocab)
test_set = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv", inp_vocab, out_vocab)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_set, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, collate_fn=collate_fn)

encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim, config.enc_layers, config.cell_type, config.dropout)
decoder = Decoder(out_vocab.size, config.embedding_dim, config.hidden_dim, config.dec_layers, config.cell_type, config.dropout)
model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers, config.cell_type, device).to(device)

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)

best_val_acc = 0.0
for epoch in range(config.epochs):
    train_loss, train_acc = train_eval(model, train_loader, criterion, optimizer, is_train=True)
    val_loss, val_acc = train_eval(model, dev_loader, criterion, optimizer, is_train=False)
    wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "epoch": epoch})
    print(f"Epoch {epoch}: Train Loss={train_loss:.4f} Acc={train_acc:.4f}, Val Loss={val_loss:.4f} Acc={val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")

# =======================
# Test Evaluation
# =======================
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

os.makedirs("predictions_vanilla", exist_ok=True)
with open("predictions_vanilla/preds.txt", "w", encoding="utf-8") as f:
    correct, total = 0, 0
    samples = []
    for src, trg, src_lens, _ in test_loader:
        src, trg = src.to(device), trg.to(device)
        output = model((src, src_lens), trg, teacher_forcing_ratio=0.0)
        pred_idxs = output.argmax(-1)[0].tolist()
        true_idxs = trg[0].tolist()
        pred_str = out_vocab.decode(pred_idxs)
        true_str = out_vocab.decode(true_idxs)
        input_str = inp_vocab.decode(src[0].tolist())
        f.write(f"{input_str}\t{true_str}\t{pred_str}\n")
        if pred_str == true_str:
            correct += 1
        total += 1
        samples.append((input_str, true_str, pred_str))
    test_acc = correct / total
    wandb.log({"test_accuracy": test_acc})
    print("Test Accuracy:", test_acc)

# Sample Grid (for visualization)
print("\nSample Predictions:")
print("{:<20} | {:<20} | {:<20}".format("Input", "Reference", "Prediction"))
print("=" * 65)
for s in random.sample(samples, 10):
    print("{:<20} | {:<20} | {:<20}".format(*s))


Epoch 0: Train Loss=1.2103 Acc=0.6221, Val Loss=0.7996 Acc=0.7455
Epoch 1: Train Loss=0.7029 Acc=0.7740, Val Loss=0.6784 Acc=0.7765
Epoch 2: Train Loss=0.6081 Acc=0.8035, Val Loss=0.6208 Acc=0.8025
Epoch 3: Train Loss=0.5603 Acc=0.8173, Val Loss=0.6208 Acc=0.8002
Epoch 4: Train Loss=0.5335 Acc=0.8238, Val Loss=0.6086 Acc=0.8038
Epoch 5: Train Loss=0.5105 Acc=0.8311, Val Loss=0.6006 Acc=0.8093
Epoch 6: Train Loss=0.4914 Acc=0.8355, Val Loss=0.5927 Acc=0.8080
Epoch 7: Train Loss=0.4737 Acc=0.8404, Val Loss=0.6052 Acc=0.8078
Epoch 8: Train Loss=0.4590 Acc=0.8435, Val Loss=0.6019 Acc=0.8034
Epoch 9: Train Loss=0.4495 Acc=0.8457, Val Loss=0.5979 Acc=0.8129
Epoch 10: Train Loss=0.4416 Acc=0.8482, Val Loss=0.6096 Acc=0.8070
Epoch 11: Train Loss=0.4337 Acc=0.8492, Val Loss=0.6045 Acc=0.8113
Epoch 12: Train Loss=0.4265 Acc=0.8510, Val Loss=0.5946 Acc=0.8153
Epoch 13: Train Loss=0.4222 Acc=0.8518, Val Loss=0.6018 Acc=0.8093
Epoch 14: Train Loss=0.4120 Acc=0.8546, Val Loss=0.6180 Acc=0.8116
Epoch

In [18]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math
from tqdm import tqdm

os.makedirs("predictions_vanilla", exist_ok=True)

best_config = {
    'embedding_dim': 128,
    'hidden_dim': 256,
    'enc_layers': 3,
    'dec_layers': 3,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 15,
    'beam_size': 5
    'batch_size': 128,  # Added
    'learning_rate': 0.001  # Added
}

# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab, is_test=False):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        if not is_test:
            inp_vocab.build([p[0] for p in self.pairs])
            out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y), lat, dev

def collate_fn(batch):
    x_batch, y_batch, lat, dev = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens), lat, dev

# =======================
# Encoder, Decoder, Seq2Seq
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])
        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)
        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

    def predict(self, src_tensor, src_len, max_len=30):
        self.eval()
        with torch.no_grad():
            enc_hidden = self.encoder(src_tensor.unsqueeze(0), torch.tensor([src_len]))
            if self.cell_type == "LSTM":
                h, c = enc_hidden
                h = self._match_layers(h)
                c = self._match_layers(c)
                dec_hidden = (h, c)
            else:
                dec_hidden = self._match_layers(enc_hidden)
            input_token = torch.tensor([2]).to(self.device)  # <sos>
            output_seq = []
            for _ in range(max_len):
                output, dec_hidden = self.decoder(input_token, dec_hidden)
                top1 = output.argmax(1)
                if top1.item() == 2: break  # <eos>
                output_seq.append(top1.item())
                input_token = top1
        return output_seq

# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _, _, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _, _, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

# =======================
# Main
# =======================
def main():
    wandb.init(project="dakshina-transliteration")
    config = best_config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv", inp_vocab, out_vacab)
    dev_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv", inp_vocab, out_vocab)
    test_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv", inp_vocab, out_vocab, is_test=True)

    # Update DataLoader batch sizes
    train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collate_fn)

    # Update optimizer with learning rate
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    best_val_acc = 0.0
    for epoch in range(config['epochs']):
        print(f"Epoch {epoch+1}/{config['epochs']}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("Best model saved.")

    
    # Evaluation on test set
    print("\n Evaluating on test set with best model:")
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    total_correct, total_count = 0, 0

    with open("predictions_vanilla/test_predictions.txt", "w", encoding="utf-8") as f:
        for src, _, src_lens, _, lat, gold in test_loader:
            src = src.to(device)
            pred_ids = model.predict(src[0], src_lens[0].item())
            pred = out_vocab.decode(pred_ids)
            f.write(f"{lat[0]}\t{gold[0]}\t{pred}\n")
            if pred == gold[0]:
                total_correct += 1
            total_count += 1

    test_acc = 100.0 * total_correct / total_count
    print(f"📊 Test Accuracy: {test_acc:.2f}%")
    wandb.log({"test_acc": test_acc})


     # After test evaluation, add random samples display
    print("\nRandom Test Samples Predictions:")
    import random
    random_indices = random.sample(range(len(test_data)), 20)
    
    with open("predictions_vanilla/test_predictions.txt", "a", encoding="utf-8") as f:
        f.write("\n\nRandom Sample Predictions:\n")
        for idx in random_indices:
            x, y, lat, dev = test_data[idx]
            src_tensor = x.to(device)
            pred_ids = model.predict(src_tensor, len(x))
            pred = out_vocab.decode(pred_ids)
            
            print(f"Input: {lat}")
            print(f"True: {dev}")
            print(f"Pred: {pred}\n")
            
            f.write(f"Input: {lat}\n")
            f.write(f"True: {dev}\n")
            f.write(f"Pred: {pred}\n\n")


if __name__ == "__main__":
    main()




train_acc,▁▆▇███
train_loss,█▃▂▂▁▁
val_acc,▁▆▇▇██
val_loss,█▄▂▂▁▂
train_acc,83.12333
train_loss,0.50879
val_acc,72.13333
val_loss,0.86891


Epoch 1/15
Train Loss: 1.2916, Accuracy: 60.52%
Val   Loss: 0.9746, Accuracy: 68.25%
✅ Best model saved.
Epoch 2/15
Train Loss: 0.6719, Accuracy: 78.24%
Val   Loss: 0.8701, Accuracy: 71.92%
✅ Best model saved.
Epoch 3/15
Train Loss: 0.5746, Accuracy: 81.27%
Val   Loss: 0.8041, Accuracy: 73.33%
✅ Best model saved.
Epoch 4/15
Train Loss: 0.5236, Accuracy: 82.72%
Val   Loss: 0.8114, Accuracy: 73.86%
✅ Best model saved.
Epoch 5/15
Train Loss: 0.4914, Accuracy: 83.66%
Val   Loss: 0.8050, Accuracy: 74.60%
✅ Best model saved.
Epoch 6/15
Train Loss: 0.4701, Accuracy: 84.21%
Val   Loss: 0.8052, Accuracy: 74.57%
Epoch 7/15
Train Loss: 0.4553, Accuracy: 84.49%
Val   Loss: 0.8100, Accuracy: 74.55%
Epoch 8/15
Train Loss: 0.4394, Accuracy: 84.89%
Val   Loss: 0.7896, Accuracy: 74.93%
✅ Best model saved.
Epoch 9/15
Train Loss: 0.4285, Accuracy: 85.05%
Val   Loss: 0.7989, Accuracy: 74.77%
Epoch 10/15
Train Loss: 0.4173, Accuracy: 85.34%
Val   Loss: 0.7853, Accuracy: 75.38%
✅ Best model saved.
Epoch 11/

In [ ]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os

sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_loss', 'goal': 'minimize'},
    'parameters': {
        'embedding_dim': {'values': [64, 128,256]},
        'hidden_dim': {'values': [64, 128,256]},
        'enc_layers': {'values': [1, 2,3]},
        'dec_layers': {'values': [1, 2,3]},
        'cell_type': {'values': ['GRU', 'LSTM', 'RNN']},
        'dropout': {'values': [0.0,0.2, 0.3]},
        'epochs': {'values': [10,15]},
        'beam_size': {'values': [1, 3, 5]}
    }
}

default_config = {
    'embedding_dim': 64,
    'hidden_dim': 64,
    'enc_layers': 1,
    'dec_layers': 1,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 10,
    'beam_size': 1
}

# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.word2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2word = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for word in text.split():
                if word not in self.word2idx:
                    self.word2idx[word] = self.size
                    self.idx2word[self.size] = word
                    self.size += 1

    def encode(self, text):
        return [self.word2idx[w] for w in text.split()]

    def decode(self, idxs):
        return ' '.join([self.idx2word[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        inp_vocab.build([p[0] for p in self.pairs])
        out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self): return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.word2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.word2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Encoder, Decoder
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        emb = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=False)
        _, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        emb = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(emb, hidden)
        return self.fc(output.squeeze(1)), hidden

# =======================
# Seq2Seq Model + Beam Search
# =======================
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.cell_type = cell_type
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.fc.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])
        dec_hidden = self._match_layers(enc_hidden)
        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def beam_search(self, src, src_len, sos_idx, eos_idx, beam_size, max_len=20):
        enc_hidden = self.encoder(src, src_len)
        dec_hidden = self._match_layers(enc_hidden)

        sequences = [[sos_idx]]
        scores = [0.0]

        for _ in range(max_len):
            all_candidates = []
            for i, seq in enumerate(sequences):
                input_token = torch.tensor([seq[-1]]).to(self.device)
                output, new_hidden = self.decoder(input_token, dec_hidden)
                probs = torch.log_softmax(output, dim=-1)
                topk = torch.topk(probs, beam_size)
                for j in range(beam_size):
                    candidate = seq + [topk.indices[0][j].item()]
                    score = scores[i] + topk.values[0][j].item()
                    all_candidates.append((candidate, score))
            ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
            sequences, scores = zip(*ordered[:beam_size])
            if all(seq[-1] == eos_idx for seq in sequences):
                break
        return sequences[0]

    def _match_layers(self, hidden):
        if self.cell_type == 'LSTM':
            h, c = hidden
            return self._pad_layers(h), self._pad_layers(c)
        return self._pad_layers(hidden)

    def _pad_layers(self, h):
        if self.enc_layers == self.dec_layers:
            return h
        elif self.enc_layers > self.dec_layers:
            return h[:self.dec_layers]
        else:
            pad = h.new_zeros((self.dec_layers - self.enc_layers, *h.shape[1:]))
            return torch.cat([h, pad], dim=0)

# =======================
# Train and Evaluate
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

def evaluate(model, loader, criterion, device, out_vocab, beam_size):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            if beam_size > 1:
                for i in range(src.size(0)):
                    pred_seq = model.beam_search(src[i].unsqueeze(0), src_lens[i:i+1], 1, 2, beam_size)
                    gold_seq = trg[i].tolist()
                    total_correct += sum(p == g for p, g in zip(pred_seq[1:], gold_seq[1:]) if g != 0)
                    total_count += sum(1 for g in gold_seq[1:] if g != 0)
                total_loss += 0  # No loss in beam search mode
            else:
                output = model((src, src_lens), trg, teacher_forcing_ratio=0)
                loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
                pred = output.argmax(2)
                correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
                total_correct += correct
                total_count += (trg[:, 1:] != 0).sum().item()
                total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

# =======================
# Main
# =======================
def main():
    wandb.init(config=default_config, project="word-level-transliteration")
    config = wandb.config
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv", inp_vocab, out_vocab)
    dev_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv", inp_vocab, out_vocab)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim, config.enc_layers, config.cell_type, config.dropout)
    decoder = Decoder(out_vocab.size, config.embedding_dim, config.hidden_dim, config.dec_layers, config.cell_type, config.dropout)
    model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers, config.cell_type, device).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(config.epochs):
        print(f"Epoch {epoch+1}/{config.epochs}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device, out_vocab, config.beam_size)
        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f} Acc={train_acc:.4f}, Val Loss={val_loss:.4f} Acc={val_acc:.4f}")

        wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "epoch": epoch+1})

# =======================
if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="word-level-transliteration")
    wandb.agent(sweep_id, function=main, count=5)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: v8nboqsd
Sweep URL: https://wandb.ai/manglesh_dl_ass3/word-level-transliteration/sweeps/v8nboqsd


wandb: Agent Starting Run: 2eokon6t with config:
wandb: 	beam_size: 3
wandb: 	cell_type: RNN
wandb: 	dec_layers: 1
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 64
wandb: 	enc_layers: 2
wandb: 	epochs: 15
wandb: 	hidden_dim: 64
wandb: Currently logged in as: manglesh_dlass3 (manglesh_dl_ass3) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/15
Epoch 1: Train Loss=5.7940 Acc=49.5804, Val Loss=0.0000 Acc=48.1758
Epoch 2/15
Epoch 2: Train Loss=5.3633 Acc=50.0000, Val Loss=0.0000 Acc=43.5865
Epoch 3/15
Epoch 3: Train Loss=5.0938 Acc=50.0181, Val Loss=0.0000 Acc=39.4103
Epoch 4/15
Epoch 4: Train Loss=4.5842 Acc=50.3699, Val Loss=0.0000 Acc=39.0776
Epoch 5/15
Epoch 5: Train Loss=3.9932 Acc=52.3686, Val Loss=0.0000 Acc=37.5516
Epoch 6/15
Epoch 6: Train Loss=3.4184 Acc=57.0390, Val Loss=0.0000 Acc=38.8596
Epoch 7/15
Epoch 7: Train Loss=2.8836 Acc=62.4333, Val Loss=0.0000 Acc=39.7774
Epoch 8/15
Epoch 8: Train Loss=2.3975 Acc=66.3096, Val Loss=0.0000 Acc=40.8215
Epoch 9/15
Epoch 9: Train Loss=1.9742 Acc=68.3739, Val Loss=0.0000 Acc=41.7393
Epoch 10/15
Epoch 10: Train Loss=1.6258 Acc=69.0944, Val Loss=0.0000 Acc=42.0262
Epoch 11/15
Epoch 11: Train Loss=1.3579 Acc=69.3105, Val Loss=0.0000 Acc=43.0014
Epoch 12/15
Epoch 12: Train Loss=1.1646 Acc=69.4926, Val Loss=0.0000 Acc=43.0702
Epoch 13/15
Epoch 13: Train Loss=1.0331 Acc=69

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▁▁▁▂▄▆▇███████
train_loss,█▇▇▆▅▅▄▃▃▂▂▁▁▁▁
val_acc,█▅▂▂▁▂▂▃▄▄▅▅▅▅▅
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,15
train_acc,69.48466
train_loss,0.88627
val_acc,43.6324
val_loss,0


wandb: Agent Starting Run: qagkeha0 with config:
wandb: 	beam_size: 5
wandb: 	cell_type: RNN
wandb: 	dec_layers: 1
wandb: 	dropout: 0.2
wandb: 	embedding_dim: 256
wandb: 	enc_layers: 1
wandb: 	epochs: 15
wandb: 	hidden_dim: 64


Epoch 1/15
Epoch 1: Train Loss=6.1236 Acc=49.6120, Val Loss=0.0000 Acc=26.2391
Epoch 2/15
Epoch 2: Train Loss=5.5682 Acc=50.0011, Val Loss=0.0000 Acc=25.8720
Epoch 3/15
Epoch 3: Train Loss=5.4350 Acc=50.0011, Val Loss=0.0000 Acc=28.7976
Epoch 4/15
Epoch 4: Train Loss=5.3200 Acc=50.0034, Val Loss=0.0000 Acc=30.7480
Epoch 5/15


In [4]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'embedding_dim': {'values': [32,64,128,256]},
        'hidden_dim': {'values': [64, 128,256 ]},
        'enc_layers': {'values': [1, 2,3]},
        'dec_layers': {'values': [1, 2,3]},
        'cell_type': {'values': ['GRU', 'LSTM', 'RNN']},
        'dropout': {'values': [0.0,0.2, 0.3,0.5]},
        'epochs': {'values': [10,20, 15]},
        'beam_size': {'values': [1, 3, 5]},
        'batch_size': {'values': [64, 128, 256]},
        'learning_rate': {'values': [0.001, 0.0005, 0.0001]}
    }
}

default_config = {
    'embedding_dim': 32,
    'hidden_dim': 64,
    'enc_layers': 1,
    'dec_layers': 1,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 10,
    'beam_size': 1,
    'batch_size': 64,
    'learning_rate': 0.001
}


# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        inp_vocab.build([p[0] for p in self.pairs])
        out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Encoder and Decoder
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

# =======================
# Seq2Seq Model with Beam Search
# =======================
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])

        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)

        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Train Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Val Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

# =======================
# Main
# =======================
def main():
    wandb.init(config=default_config, project="dakshina-transliteration")
    config = wandb.config
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
    dev_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
    train_data = TransliterationDataset(train_path, inp_vocab, out_vocab)
    dev_data = TransliterationDataset(dev_path, inp_vocab, out_vocab)
    
    # Use config.batch_size for DataLoader
    train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim, config.enc_layers, config.cell_type, config.dropout)
    decoder = Decoder(out_vocab.size, config.embedding_dim, config.hidden_dim, config.dec_layers, config.cell_type, config.dropout)
    model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers, config.cell_type, device).to(device)
    
    # Use config.learning_rate for optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(config.epochs):
        print(f"Epoch {epoch+1}/{config.epochs}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "epoch": epoch+1})

# =======================

if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="dakshina-translit")
    wandb.agent(sweep_id, function=main,count=30)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 2yk21c98
Sweep URL: https://wandb.ai/manglesh_dl_ass3/dakshina-translit/sweeps/2yk21c98


wandb: Agent Starting Run: f8lsoyx5 with config:
wandb: 	batch_size: 128
wandb: 	beam_size: 3
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 2
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 128
wandb: 	enc_layers: 1
wandb: 	epochs: 15
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: manglesh_dlass3 (manglesh_dl_ass3) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/15
Train Loss: 2.8135, Acc: 23.51%
Val Loss: 2.5705, Acc: 28.12%
Epoch 2/15
Train Loss: 2.4345, Acc: 29.73%
Val Loss: 2.3086, Acc: 30.87%
Epoch 3/15
Train Loss: 2.2041, Acc: 32.47%
Val Loss: 2.1289, Acc: 32.56%
Epoch 4/15
Train Loss: 2.0241, Acc: 35.56%
Val Loss: 2.0125, Acc: 34.26%
Epoch 5/15
Train Loss: 1.8849, Acc: 38.74%
Val Loss: 1.9114, Acc: 36.35%
Epoch 6/15
Train Loss: 1.7606, Acc: 42.41%
Val Loss: 1.8224, Acc: 38.88%
Epoch 7/15
Train Loss: 1.6551, Acc: 45.66%
Val Loss: 1.7421, Acc: 41.13%
Epoch 8/15
Train Loss: 1.5627, Acc: 48.60%
Val Loss: 1.6578, Acc: 44.00%
Epoch 9/15
Train Loss: 1.4863, Acc: 50.87%
Val Loss: 1.5707, Acc: 47.06%
Epoch 10/15
Train Loss: 1.4028, Acc: 53.94%
Val Loss: 1.4997, Acc: 49.84%
Epoch 11/15
Train Loss: 1.3305, Acc: 56.48%
Val Loss: 1.4181, Acc: 52.45%
Epoch 12/15
Train Loss: 1.2608, Acc: 58.98%
Val Loss: 1.3476, Acc: 55.49%
Epoch 13/15
Train Loss: 1.2049, Acc: 60.94%
Val Loss: 1.2937, Acc: 56.91%
Epoch 14/15
Train Loss: 1.1443, Acc: 63.20%
Val

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▂▃▃▄▄▅▅▆▆▇▇▇██
train_loss,█▆▆▅▄▄▃▃▃▂▂▂▁▁▁
val_acc,▁▂▂▂▃▃▄▄▅▆▆▇▇██
val_loss,█▇▆▅▅▄▄▃▃▃▂▂▁▁▁
epoch,15
train_acc,64.67042
train_loss,1.09736
val_acc,60.21238
val_loss,1.19925


...

Epoch 1/15
Train Loss: 1.2661, Acc: 59.65%
Val Loss: 1.0002, Acc: 66.62%
Epoch 2/15
Train Loss: 0.7886, Acc: 74.16%
Val Loss: 0.8904, Acc: 70.81%
Epoch 3/15
Train Loss: 0.6842, Acc: 77.68%
Val Loss: 0.8809, Acc: 71.32%
Epoch 4/15
Train Loss: 0.6292, Acc: 79.39%
Val Loss: 0.8741, Acc: 72.36%
Epoch 5/15
Train Loss: 0.5966, Acc: 80.38%
Val Loss: 0.8798, Acc: 73.05%
Epoch 6/15
Train Loss: 0.5733, Acc: 81.13%
Val Loss: 0.8426, Acc: 72.72%
Epoch 7/15
Train Loss: 0.5643, Acc: 81.36%
Val Loss: 0.8669, Acc: 73.15%
Epoch 8/15
Train Loss: 0.5520, Acc: 81.72%
Val Loss: 0.8347, Acc: 73.32%
Epoch 9/15
Train Loss: 0.5412, Acc: 82.00%
Val Loss: 0.8197, Acc: 73.34%
Epoch 10/15
Train Loss: 0.5219, Acc: 82.68%
Val Loss: 0.8537, Acc: 73.57%
Epoch 11/15
Train Loss: 0.5224, Acc: 82.59%
Val Loss: 0.8548, Acc: 73.11%
Epoch 12/15
Train Loss: 0.5123, Acc: 82.92%
Val Loss: 0.8518, Acc: 73.37%
Epoch 13/15
Train Loss: 0.5099, Acc: 82.93%
Val Loss: 0.8633, Acc: 73.76%
Epoch 14/15
Train Loss: 0.4981, Acc: 83.23%
Val

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▅▆▇▇▇▇████████
train_loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▅▅▆▇▇▇▇▇█▇▇███
val_loss,█▄▃▃▃▂▃▂▁▂▂▂▃▂▁
epoch,15
train_acc,83.12333
train_loss,0.5026
val_acc,74.08929
val_loss,0.81907


wandb: Agent Starting Run: tg4tt48f with config:
wandb: 	batch_size: 64
wandb: 	beam_size: 1
wandb: 	cell_type: GRU
wandb: 	dec_layers: 3
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 256
wandb: 	enc_layers: 2
wandb: 	epochs: 15
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.001


Epoch 1/15
Train Loss: 1.4077, Acc: 54.87%
Val Loss: 1.0090, Acc: 66.14%
Epoch 2/15
Train Loss: 0.8640, Acc: 71.54%
Val Loss: 0.9221, Acc: 69.50%
Epoch 3/15
Train Loss: 0.7513, Acc: 75.29%
Val Loss: 0.8711, Acc: 71.03%
Epoch 4/15
Train Loss: 0.6924, Acc: 77.31%
Val Loss: 0.8669, Acc: 71.88%
Epoch 5/15
Train Loss: 0.6581, Acc: 78.29%
Val Loss: 0.8526, Acc: 72.66%
Epoch 6/15
Train Loss: 0.6410, Acc: 78.81%
Val Loss: 0.8505, Acc: 73.46%
Epoch 7/15
Train Loss: 0.6177, Acc: 79.61%
Val Loss: 0.8647, Acc: 73.26%
Epoch 8/15
Train Loss: 0.6004, Acc: 80.14%
Val Loss: 0.8810, Acc: 73.43%
Epoch 9/15
Train Loss: 0.5909, Acc: 80.56%
Val Loss: 0.8552, Acc: 73.15%
Epoch 10/15
Train Loss: 0.5896, Acc: 80.41%
Val Loss: 0.8119, Acc: 73.61%
Epoch 11/15
Train Loss: 0.5795, Acc: 80.69%
Val Loss: 0.8255, Acc: 73.75%
Epoch 12/15
Train Loss: 0.5674, Acc: 81.18%
Val Loss: 0.8329, Acc: 73.74%
Epoch 13/15
Train Loss: 0.5686, Acc: 81.12%
Val Loss: 0.8025, Acc: 74.07%
Epoch 14/15
Train Loss: 0.5660, Acc: 81.03%
Val

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▅▆▇▇▇▇████████
train_loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▄▅▆▇▇▇▇▇▇▇▇███
val_loss,█▅▃▃▃▃▃▄▃▁▂▂▁▂▂
epoch,15
train_acc,81.72257
train_loss,0.54951
val_acc,74.37863
val_loss,0.81987


In [10]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math
from tqdm import tqdm

os.makedirs("predictions_vanilla", exist_ok=True)

best_config = {
    'embedding_dim': 128,
    'hidden_dim': 256,
    'enc_layers': 2,
    'dec_layers': 3,
    'cell_type': 'LSTM',
    'dropout': 0.5,
    'epochs': 15,
    'beam_size': 3,
    'batch_size': 64,  # Added
    'learning_rate': 0.0005  # Added
}

# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab, is_test=False):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        if not is_test:
            inp_vocab.build([p[0] for p in self.pairs])
            out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y), lat, dev

def collate_fn(batch):
    x_batch, y_batch, lat, dev = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens), lat, dev

# =======================
# Encoder, Decoder, Seq2Seq
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])
        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)
        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

    def predict(self, src_tensor, src_len, max_len=30):
        self.eval()
        with torch.no_grad():
            enc_hidden = self.encoder(src_tensor.unsqueeze(0), torch.tensor([src_len]))
            if self.cell_type == "LSTM":
                h, c = enc_hidden
                h = self._match_layers(h)
                c = self._match_layers(c)
                dec_hidden = (h, c)
            else:
                dec_hidden = self._match_layers(enc_hidden)
            input_token = torch.tensor([2]).to(self.device)  # <sos>
            output_seq = []
            for _ in range(max_len):
                output, dec_hidden = self.decoder(input_token, dec_hidden)
                top1 = output.argmax(1)
                if top1.item() == 2: break  # <eos>
                output_seq.append(top1.item())
                input_token = top1
        return output_seq

# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _, _, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _, _, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

# =======================
# Main
# =======================
def main():
    wandb.init(project="dakshina-translit-test")
    config = best_config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv", inp_vocab, out_vocab)
    dev_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv", inp_vocab, out_vocab)
    test_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv", inp_vocab, out_vocab, is_test=True)

    # Update DataLoader batch sizes
    train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collate_fn)
     # First create model
    encoder = Encoder(inp_vocab.size, config['embedding_dim'], config['hidden_dim'],
                      config['enc_layers'], config['cell_type'], config['dropout'])
    decoder = Decoder(out_vocab.size, config['embedding_dim'], config['hidden_dim'],
                      config['dec_layers'], config['cell_type'], config['dropout'])
    model = Seq2Seq(encoder, decoder, config['enc_layers'], config['dec_layers'], config['cell_type'], device).to(device)
    # Update optimizer with learning rate
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    best_val_acc = 0.0
    for epoch in range(config['epochs']):
        print(f"Epoch {epoch+1}/{config['epochs']}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("Best model saved.")

    
    # Evaluation on test set
    print("\n Evaluating on test set with best model:")
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    total_correct, total_count = 0, 0

    with open("predictions_vanilla/test_predictions.txt", "w", encoding="utf-8") as f:
        for src, _, src_lens, _, lat, gold in test_loader:
            src = src.to(device)
            pred_ids = model.predict(src[0], src_lens[0].item())
            pred = out_vocab.decode(pred_ids)
            f.write(f"{lat[0]}\t{gold[0]}\t{pred}\n")
            if pred == gold[0]:
                total_correct += 1
            total_count += 1

    test_acc = 100.0 * total_correct / total_count
    print(f"📊 Test Accuracy: {test_acc:.2f}%")
    wandb.log({"test_acc": test_acc})


     # After test evaluation, add random samples display
    print("\nRandom Test Samples Predictions:")
    import random
    random_indices = random.sample(range(len(test_data)), 30)
    
    with open("predictions_vanilla/test_predictions.txt", "a", encoding="utf-8") as f:
        f.write("\n\nRandom Sample Predictions:\n")
        for idx in random_indices:
            x, y, lat, dev = test_data[idx]
            src_tensor = x.to(device)
            pred_ids = model.predict(src_tensor, len(x))
            pred = out_vocab.decode(pred_ids)
            
            print(f"Input: {lat}")
            print(f"True: {dev}")
            print(f"Pred: {pred}\n")
            
            f.write(f"Input: {lat}\n")
            f.write(f"True: {dev}\n")
            f.write(f"Pred: {pred}\n\n")


if __name__ == "__main__":
    main()




Epoch 1/15
Train Loss: 1.7807, Accuracy: 47.10%
Val   Loss: 1.0987, Accuracy: 62.92%
Best model saved.
Epoch 2/15
Train Loss: 0.8313, Accuracy: 73.17%
Val   Loss: 0.8883, Accuracy: 69.75%
Best model saved.
Epoch 3/15
Train Loss: 0.6634, Accuracy: 78.55%
Val   Loss: 0.8675, Accuracy: 71.87%
Best model saved.
Epoch 4/15
Train Loss: 0.5937, Accuracy: 80.57%
Val   Loss: 0.8368, Accuracy: 72.91%
Best model saved.
Epoch 5/15
Train Loss: 0.5427, Accuracy: 82.14%
Val   Loss: 0.8239, Accuracy: 73.44%
Best model saved.
Epoch 6/15
Train Loss: 0.5105, Accuracy: 83.07%
Val   Loss: 0.8186, Accuracy: 73.39%
Epoch 7/15
Train Loss: 0.4891, Accuracy: 83.70%
Val   Loss: 0.7836, Accuracy: 74.55%
Best model saved.
Epoch 8/15
Train Loss: 0.4709, Accuracy: 84.08%
Val   Loss: 0.7878, Accuracy: 74.21%
Epoch 9/15
Train Loss: 0.4516, Accuracy: 84.61%
Val   Loss: 0.7778, Accuracy: 74.79%
Best model saved.
Epoch 10/15
Train Loss: 0.4297, Accuracy: 85.32%
Val   Loss: 0.8025, Accuracy: 74.56%
Epoch 11/15
Train Loss: